<a href="https://colab.research.google.com/github/kyungmin0101/medical_ai/blob/main/240221/Medical_Ai_day_6_Hyunsang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import glob
import csv
import pandas as pd

#All of 1stSeries data
path1 = "/content/drive/MyDrive/Refractive Index Measurements New/1stSeries_07.05.21/*/{}*".format('W') #training set
df_list_1st = []
ri_list_1st = []
for file in glob.glob(path1, recursive = True) :
  f = open(file,'rt')
  reader = csv.reader(f)
  csv_context = []
  for raw in reader :
    csv_context.append(raw)
  f.close()

  df_list_1st.append(pd.DataFrame(csv_context)[24:])
  ri_list_1st.append(file.split('/')[-2])

#All of 2ndSeries data
path2 = "/content/drive/MyDrive/Refractive Index Measurements New/2ndSeries_21.05.21/*/{}*".format('W')#test set
df_list_2nd = []
ri_list_2nd = []
for file in glob.glob(path2, recursive = True) :
  f = open(file,'rt')
  reader = csv.reader(f)
  csv_context = []
  for raw in reader :
    csv_context.append(raw)
  f.close()

  df_list_2nd.append(pd.DataFrame(csv_context)[24:])
  ri_list_2nd.append(file.split('/')[-2])

In [4]:
ri_list_1st_CIN=[]
for i in range(len(ri_list_1st)) :
  if 1.3 <= float(ri_list_1st[i]) <= 1.32 :
    ri_list_1st_CIN.append("CIN1")
  if 1.33 <= float(ri_list_1st[i]) <= 1.37 :
    ri_list_1st_CIN.append("CIN2")
  if 1.38 <= float(ri_list_1st[i]) <= 1.39 :
    ri_list_1st_CIN.append("CIN3")
  if float(ri_list_1st[i]) >= 1.4 :
    ri_list_1st_CIN.append("Cervical Cancer")

ri_list_2nd_CIN=[]
for i in range(len(ri_list_2nd)) :
  if 1.3 <= float(ri_list_2nd[i]) <= 1.32 :
    ri_list_2nd_CIN.append("CIN1")
  if 1.33 <= float(ri_list_2nd[i]) <= 1.37 :
    ri_list_2nd_CIN.append("CIN2")
  if 1.38 <= float(ri_list_2nd[i]) <= 1.39 :
    ri_list_2nd_CIN.append("CIN3")
  if float(ri_list_2nd[i]) >= 1.4 :
    ri_list_2nd_CIN.append("Cervical Cancer")

In [54]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
import numpy as np
import time

x_train_df_list_2nd, x_test_df_list_2nd, y_train_ri_list_2nd, y_test_ri_list_2nd_CIN = train_test_split(df_list_2nd, ri_list_2nd_CIN, test_size=0.1)

nsamples, nx, ny = np.array(df_list_1st).shape
x_train_df_list_1st = np.array(df_list_1st).reshape((nsamples,nx*ny))

nsamples, nx, ny = np.array(x_test_df_list_2nd).shape
x_test_df_list_2nd = np.array(x_test_df_list_2nd).reshape((nsamples,nx*ny))

#Tree_model
tree_clf = DecisionTreeClassifier(random_state=50)
tree_clf.fit(x_train_df_list_1st, ri_list_1st_CIN)
#KNN
knn_clf = KNeighborsClassifier(n_neighbors = 5)
knn_clf.fit(x_train_df_list_1st, ri_list_1st_CIN)
#SVM
svm_clf = svm.SVC(kernel='linear', C=1,probability=True)
svm_clf.fit(x_train_df_list_1st, ri_list_1st_CIN)


SVC(C=1, kernel='linear', probability=True)

In [55]:
tree_pred =tree_clf.predict(x_test_df_list_2nd)
knn_pred = knn_clf.predict(x_test_df_list_2nd)
svm_pred = svm_clf.predict(x_test_df_list_2nd)

print('tree Acc : {}'.format(accuracy_score(y_test_ri_list_2nd_CIN,tree_pred)))
print(classification_report(y_test_ri_list_2nd_CIN,tree_pred))
print('knn Acc : {}'.format(accuracy_score(y_test_ri_list_2nd_CIN,knn_pred)))
print(classification_report(y_test_ri_list_2nd_CIN,knn_pred))
print('svm Acc : {}'.format(accuracy_score(y_test_ri_list_2nd_CIN,svm_pred)))
print(classification_report(y_test_ri_list_2nd_CIN,svm_pred))

tree Acc : 0.125
                 precision    recall  f1-score   support

           CIN1       0.00      0.00      0.00         4
           CIN2       0.10      1.00      0.17         2
           CIN3       0.00      0.00      0.00         3
Cervical Cancer       1.00      0.07      0.12        15

       accuracy                           0.12        24
      macro avg       0.27      0.27      0.07        24
   weighted avg       0.63      0.12      0.09        24

knn Acc : 0.2916666666666667
                 precision    recall  f1-score   support

           CIN1       0.00      0.00      0.00         4
           CIN2       0.11      1.00      0.20         2
           CIN3       0.00      0.00      0.00         3
Cervical Cancer       1.00      0.33      0.50        15

       accuracy                           0.29        24
      macro avg       0.28      0.33      0.17        24
   weighted avg       0.63      0.29      0.33        24

svm Acc : 0.625
                 pre

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [56]:
##ensembel
from sklearn.ensemble import VotingClassifier

ensemble_model = VotingClassifier(estimators=[('tree', tree_clf), ('knn', knn_clf), ('svm', svm_clf)], voting='soft')
ensemble_model.fit(x_train_df_list_1st, ri_list_1st_CIN)

predictions = ensemble_model.predict(x_test_df_list_2nd)

print('Voting Acc : {}'.format(accuracy_score(y_test_ri_list_2nd_CIN,predictions)))
print(classification_report(y_test_ri_list_2nd_CIN,predictions))


Voting Acc : 0.125
                 precision    recall  f1-score   support

           CIN1       0.00      0.00      0.00         4
           CIN2       0.10      1.00      0.17         2
           CIN3       0.00      0.00      0.00         3
Cervical Cancer       1.00      0.07      0.12        15

       accuracy                           0.12        24
      macro avg       0.27      0.27      0.07        24
   weighted avg       0.63      0.12      0.09        24



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
#Bagging
from sklearn.ensemble import BaggingClassifier

tree_bagging_model = BaggingClassifier(tree_clf, n_estimators=10, random_state=42)
knn_bagging_model = BaggingClassifier(knn_clf, n_estimators=10, random_state=42)
svm_bagging_model = BaggingClassifier(svm_clf, n_estimators=10, random_state=42)

# 모델 훈련
tree_bagging_model.fit(x_train_df_list_1st, ri_list_1st_CIN)
knn_bagging_model.fit(x_train_df_list_1st, ri_list_1st_CIN)
svm_bagging_model.fit(x_train_df_list_1st, ri_list_1st_CIN)

# 예측
tree_predictions = tree_bagging_model.predict(x_test_df_list_2nd)
knn_predictions = knn_bagging_model.predict(x_test_df_list_2nd)
svm_predictions = svm_bagging_model.predict(x_test_df_list_2nd)

#결과
print('Bagging tree Acc : {}'.format(accuracy_score(y_test_ri_list_2nd_CIN,tree_pred)))
print(classification_report(y_test_ri_list_2nd_CIN,tree_predictions))
print('Bagging knn Acc : {}'.format(accuracy_score(y_test_ri_list_2nd_CIN,knn_pred)))
print(classification_report(y_test_ri_list_2nd_CIN,knn_predictions))
print('Bagging svm Acc : {}'.format(accuracy_score(y_test_ri_list_2nd_CIN,svm_pred)))
print(classification_report(y_test_ri_list_2nd_CIN,svm_predictions))

Bagging tree Acc : 0.125
                 precision    recall  f1-score   support

           CIN1       0.00      0.00      0.00         4
           CIN2       0.09      1.00      0.17         2
           CIN3       0.00      0.00      0.00         3
Cervical Cancer       1.00      0.07      0.12        15

       accuracy                           0.12        24
      macro avg       0.27      0.27      0.07        24
   weighted avg       0.63      0.12      0.09        24

Bagging knn Acc : 0.2916666666666667
                 precision    recall  f1-score   support

           CIN1       0.00      0.00      0.00         4
           CIN2       0.11      1.00      0.20         2
           CIN3       0.00      0.00      0.00         3
Cervical Cancer       1.00      0.33      0.50        15

       accuracy                           0.29        24
      macro avg       0.28      0.33      0.17        24
   weighted avg       0.63      0.29      0.33        24

Bagging svm Acc : 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [65]:
##GridSearch
from sklearn.model_selection import GridSearchCV

dtree = DecisionTreeClassifier()

grid_parameters = {"max_depth": [1, 2, 3],
                   "min_samples_split": [2, 3]
                   }

grid_dtree = GridSearchCV(dtree, param_grid=grid_parameters, cv=3, refit=True)

grid_dtree.fit(x_train_df_list_1st, ri_list_1st_CIN)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]})

In [71]:
scores_df = pd.DataFrame(grid_dtree.cv_results_)
print(scores_df)

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.933504      0.304451         0.295906        0.060640   
1       0.626400      0.181645         0.167324        0.060591   
2       0.462585      0.003530         0.120965        0.000932   
3       0.464760      0.010145         0.125091        0.006229   
4       0.491285      0.011139         0.130874        0.014152   
5       0.839529      0.015483         0.243399        0.032374   

  param_max_depth param_min_samples_split  \
0               1                       2   
1               1                       3   
2               2                       2   
3               2                       3   
4               3                       2   
5               3                       3   

                                     params  split0_test_score  \
0  {'max_depth': 1, 'min_samples_split': 2}           0.740260   
1  {'max_depth': 1, 'min_samples_split': 3}           0.740260   
2  {'max_depth':

In [72]:
#best score
print(f"Best Param: {grid_dtree.best_params_}")
print(f"Best ACC: {grid_dtree.best_score_}")


Best Param: {'max_depth': 3, 'min_samples_split': 2}
Best ACC: 0.8051948051948052
